In [7]:
import sys
sys.path.append('..')
# reload local packages automatically
%load_ext autoreload
%autoreload 2

from jax import config
config.update("jax_enable_x64", True)

# Import packages.
import cvxpy as cp
import numpy as np
import scipy

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
# Generate a random SDP.
n = 3
p = 3
np.random.seed(1)
C = np.random.randn(n, n)
A = []
b = []
for i in range(p):
    A.append(np.random.randn(n, n))
    b.append(np.random.randn())

# Define and solve the CVXPY problem.
# Create a symmetric matrix variable.
X = cp.Variable((n,n), PSD=True)
# The operator >> denotes matrix inequality.
constraints = [] # [X >> 0]
constraints += [
    cp.trace(A[i] @ X) == b[i] for i in range(p)
]
prob = cp.Problem(cp.Minimize(cp.trace(C @ X)),
                  constraints)
prob.solve(verbose=True)

# Print result.
print("The optimal value is", prob.value)
print("A solution X is")
print(X.value)

                                     CVXPY                                     
                                     v1.3.1                                    
(CVXPY) Jul 10 02:16:00 PM: Your problem has 9 variables, 3 constraints, and 0 parameters.
(CVXPY) Jul 10 02:16:00 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jul 10 02:16:00 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jul 10 02:16:00 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jul 10 02:16:00 PM: Compiling problem (target solver=SCS).
(CVXPY) Jul 10 02:16:00 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> S

------------------------------------------------------------------
	       SCS v3.2.3 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 6, constraints m: 9
cones: 	  z: primal zero / dual free vars: 3
	  s: psd vars: 6, ssize: 1
settings: eps_abs: 1.0e-05, eps_rel: 1.0e-05, eps_infeas: 1.0e-07
	  alpha: 1.50, scale: 1.00e-01, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, rho_x: 1.00e-06
	  acceleration_lookback: 10, acceleration_interval: 10
lin-sys:  sparse-direct-amd-qdldl
	  nnz(A): 24, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 3.08e+01  3.02e+00  6.83e+01 -3.09e+01  1.00e-01  4.23e-03 
   125| 9.31e-07  2.37e-07  1.86e-06  2.65e+00  5.17e-01  3.69e-02 
----------------------------------

In [ ]:
# with symmetric and >> (took 1.338e-02 seconds).

In [8]:
from opentn.transformations import create_kitaev_liouvillians, exp_operator_dt, factorize_psd, super2choi
d, N = 2, 4
dim = d**N
Lvec, Lvec_odd, Lvec_even, Lnn = create_kitaev_liouvillians(N=N, d=d, gamma=1e-2)
tau = 4
superops_exp = []
for i, op in enumerate([Lvec, Lvec_odd, Lvec_even, Lnn]):
    if i == 1 or i == 3:
        superops_exp.append(exp_operator_dt(op, tau/2, 'jax'))
    else:
        superops_exp.append(exp_operator_dt(op, tau, 'jax'))
exp_Lvec, exp_Lvec_odd, exp_Lvec_even, exp_Lnn = superops_exp

X1 = factorize_psd(psd=super2choi(exp_Lvec_odd))
X2 = factorize_psd(psd=super2choi(exp_Lvec_even))

from opentn.optimization import gds, frobenius_norm, model_Ys, compute_loss
cost_n4, grads_n4, xi_n4 = gds(fn=compute_loss, x0=[X1, X2, X1], iter=25, loss_fn=frobenius_norm, model=model_Ys, rate=1.5e-9, exact = exp_Lvec, show_cost=False)
cost_n4[-1]

DeviceArray(2.15352749e-07, dtype=float64)

In [13]:
# now that we have the other two, lets use it for an optimization of Y1
from opentn.optimization import model_Zs

X2 = xi_n4[-1][1]
X3 = xi_n4[-1][2]
Y2 = X2@X2.conj().T
Y3 = X3@X3.conj().T

# Preliminaries for CVXPY test:
- implement choi matrix composition in two different ways and see if they are equivalent

In [4]:
# lets test the two methods for choi composition
from opentn.transformations import link_product, choi_composition

# lets do the link product of Y2 and Y3
Y_23 = link_product(C1=Y2, C2=Y3, dim=dim)
# now lets the same contractions with tensordot
Y_23_tensor = choi_composition(C1=Y2, C2=Y3, dim=dim)
np.allclose(Y_23, Y_23_tensor)

True

since this worked, now the plan is to take advantage of the built in functions from CVXPY that can handle the operations in link_product which (sadly) cannot be done with choi_composition since CVXPY does not support >2D arrays

In [23]:
from opentn.transformations import super2choi
from opentn.optimization import small2zero
from scipy import sparse
# firs thing to do would to gather SDP and min square syntax:
rhs = np.array(super2choi(superop=exp_Lvec, dim=dim))
tol = 1e-8
srhs = sparse.csr_matrix(small2zero(rhs, tol)) # 3808 -> 2400
print(srhs.shape)
A = Y3@Y2
sA = sparse.csr_matrix(small2zero(A,tol)) # 19044 -> 1440
print(sA.shape)
I = sparse.identity(dim)
AxI = sparse.kron(sA, I) # 304704 -> 23040
print(AxI.shape)

(256, 256)
(256, 256)
(4096, 4096)


In [99]:
rhs.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False

In [100]:
# The operator >> denotes matrix inequality.
X = cp.Variable((dim**2,dim**2), symmetric=True)
constraints = [X >> 0]
# lets make a lhs without partial_transpose
# lhs =  cp.partial_trace(cp.kron(I, cp.partial_transpose(X, dims=[dim, dim], axis=0)) @ AxI, dims=[dim, dim, dim], axis=1)
lhs =  cp.partial_trace(cp.kron(I, X) @ AxI, dims=[dim, dim, dim], axis=1)
cost = cp.sum_squares(lhs - srhs)
prob = cp.Problem(cp.Minimize(cost), constraints)

In [7]:
def link_product_cvxpy(C1, C2, dim):
    I = sparse.identity(dim)
    C1 = sparse.csc_matrix(C1)
    C2 = sparse.csc_matrix(C2)
    C_12_expr = cp.partial_trace(cp.kron(I, cp.partial_transpose(C1, dims=[dim, dim], axis=0)) @ sparse.kron(C2, I), dims=[dim, dim, dim], axis=1)
    return C_12_expr.value #it is bound to be numpy array from cp.kron function

In [65]:
from opentn.transformations import choi_composition, link_product

np.allclose(choi_composition(X1, X2, dim), link_product_cvxpy(X1, X2, dim)) # way slower than link_product btw

True

In [38]:
A_cp = cp.partial_transpose(sA, dims=[dim, dim], axis=1)
A_cp.value

<256x256 sparse matrix of type '<class 'numpy.complex128'>'
	with 19044 stored elements in Compressed Sparse Column format>

In [40]:
A_np = sparse.csr_matrix(A.reshape([dim]*4).transpose((0,3,2,1)).reshape([dim**2, dim**2]))
np.allclose(A_np.todense(),A_cp.value.todense())

True

In [67]:
# import mosek # this was producing some errors actually

In [101]:
print(cp.installed_solvers())

['ECOS', 'ECOS_BB', 'MOSEK', 'OSQP', 'SCIPY', 'SCS']


In [102]:
prob.solve(solver=cp.MOSEK, verbose=True, canon_backend=cp.SCIPY_CANON_BACKEND)

                                     CVXPY                                     
                                     v1.3.1                                    
(CVXPY) Jul 14 03:54:13 PM: Your problem has 65536 variables, 1 constraints, and 0 parameters.
(CVXPY) Jul 14 03:54:13 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jul 14 03:54:13 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jul 14 03:54:13 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jul 14 03:54:13 PM: Compiling problem (target solver=MOSEK).
(CVXPY) Jul 14 03:54:13 PM: Reduction chain: Complex2Real -> Dcp2Cone -> CvxAttr2Constr -> C

(CVXPY) Jul 14 03:54:13 PM: Applying reduction Dcp2Cone
(CVXPY) Jul 14 03:54:13 PM: Applying reduction CvxAttr2Constr
(CVXPY) Jul 14 03:54:14 PM: Applying reduction ConeMatrixStuffing


KeyboardInterrupt: 

In [10]:
import mosek, sys

In [11]:
env = mosek.Env()
env.set_Stream(mosek.streamtype.log, sys.stdout.write)
env.echointro(1)


MOSEK Version 10.1.4(BETA) (Build date: 2023-6-26 10:07:47)
Copyright (c) MOSEK ApS, Denmark WWW: mosek.com
Platform: MACOSX/64-X86

FlexLM
 Version                  : 11.18
 Hostname                 : Cristians-Air-2
 Host ID                  : 3ca6f6546a0e
 License path             : /Users/emiliano_gr/mosek/mosek.lic

Operating system variables
 DYLD_LIBRARY_PATH        :


In [22]:
rho = np.zeros(shape=(4,4))
rho[0,0], rho[0,-1], rho[-1,0], rho[-1,-1] = 0.5, 0.5, 0.5, 0.5
print(rho)
eig, eigvecs = scipy.linalg.eigh(rho)

print(eig)
print(eigvecs)

print(rho.reshape(2,2,2,2).trace(axis1=0,axis2=2))

[[0.5 0.  0.  0.5]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]
 [0.5 0.  0.  0.5]]
[0.00000000e+00 0.00000000e+00 5.55111512e-16 1.00000000e+00]
[[ 0.          0.          0.70710678 -0.70710678]
 [ 0.         -1.          0.          0.        ]
 [ 1.          0.          0.          0.        ]
 [ 0.          0.         -0.70710678 -0.70710678]]
[[0.5 0. ]
 [0.  0.5]]


In [6]:
eig

array([0.00000000e+00, 0.00000000e+00, 5.55111512e-16, 1.00000000e+00])

In [21]:
dim = 2
rho_TB = np.reshape(rho, [dim]*4).swapaxes(0,2).reshape([dim**2, dim**2])  
print(rho_TB)
eig_TB, eigvecs_TB = scipy.linalg.eigh(rho_TB)

print(eig_TB)
print(eigvecs_TB)

print(rho_TB.reshape(2,2,2,2).trace(axis1=0,axis2=2))

[[0.5 0.  0.  0. ]
 [0.  0.  0.5 0. ]
 [0.  0.5 0.  0. ]
 [0.  0.  0.  0.5]]
[-0.5  0.5  0.5  0.5]
[[ 0.          1.          0.          0.        ]
 [ 0.70710678  0.          0.70710678  0.        ]
 [-0.70710678  0.          0.70710678  0.        ]
 [ 0.          0.          0.          1.        ]]
[[0.5 0. ]
 [0.  0.5]]


In [27]:
test = cp.Variable((dim,dim), PSD=True)
test_expr = cp.kron(test, I) @ cp.kron(I, test)
print(test_expr.is_convex())
test_cost = cp.sum_squares(test_expr - srhs)
test_prob = cp.Problem(cp.Minimize(test_cost))

False


In [28]:
test_prob.solve(solver=cp.SCS, verbose=True, canon_backend=cp.SCIPY_CANON_BACKEND)

                                     CVXPY                                     
                                     v1.3.1                                    
(CVXPY) Jul 18 09:32:02 PM: Your problem has 256 variables, 0 constraints, and 0 parameters.
(CVXPY) Jul 18 09:32:02 PM: It is compliant with the following grammars: 
(CVXPY) Jul 18 09:32:02 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jul 18 09:32:02 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.


DCPError: Problem does not follow DCP rules. Specifically:
The objective is not DCP. Its following subexpressions are not:
kron(var52,   (0, 0)	1.0
  (1, 1)	1.0
  (2, 2)	1.0
  (3, 3)	1.0
  (4, 4)	1.0
  (5, 5)	1.0
  (6, 6)	1.0
  (7, 7)	1.0
  (8, 8)	1.0
  (9, 9)	1.0
  (10, 10)	1.0
  (11, 11)	1.0
  (12, 12)	1.0
  (13, 13)	1.0
  (14, 14)	1.0
  (15, 15)	1.0) @ kron(  (0, 0)	1.0
  (1, 1)	1.0
  (2, 2)	1.0
  (3, 3)	1.0
  (4, 4)	1.0
  (5, 5)	1.0
  (6, 6)	1.0
  (7, 7)	1.0
  (8, 8)	1.0
  (9, 9)	1.0
  (10, 10)	1.0
  (11, 11)	1.0
  (12, 12)	1.0
  (13, 13)	1.0
  (14, 14)	1.0
  (15, 15)	1.0, var52)